<a href="https://colab.research.google.com/github/LlakmalGamage/English-to-Sinhala-Machine-Translation-Deep-Learning-Mini-Project-03/blob/main/translator_English_Sinhala_version_01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Loading

In [1]:
import pandas as pd

In [7]:
import pandas as pd
dataset_path="dataset_version_1.csv"


In [8]:
df1=pd.read_csv(dataset_path)


In [9]:
import random
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow as tf
import string
import re
import csv



In [10]:
df1.head(20)

,english,sinhala
0,Go.,යන්න.
1,Hi.,හායි.
2,Run!,දුවන්න!
3,Run.,දුවන්න.
4,Who?,කවුද?
5,Fire!,ගිනි!
6,Help!,උදව්!
7,Jump!,පනින්න!
8,Jump.,පනින්න.
9,Stop!,නවත්වන්න!


In [11]:
first_column_values_last_10_rows=df1['english'].tail(10)


paragraph = '\n'.join(first_column_values_last_10_rows)
# Print the paragraph
print(paragraph)

You can't view Flash content on an iPad. However, you can easily email yourself the URLs of these web pages and view that content on your regular computer when you get home.
A mistake young people often make is to start learning too many languages at the same time, as they underestimate the difficulties and overestimate their own ability to learn them.
No matter how much you try to convince people that chocolate is vanilla, it'll still be chocolate, even though you may manage to convince yourself and a few others that it's vanilla.
In 1969, Roger Miller recorded a song called "You Don't Want My Love." Today, this song is better known as "In the Summer Time." It's the first song he wrote and sang that became popular.
A child who is a native speaker usually knows many things about his or her language that a non-native speaker who has been studying for years still does not know and perhaps will never know.
There are four main causes of alcohol-related death. Injury from car accidents or v

In [12]:
len(df1)

28906

# Split the English and Sinhala translation pairs

In [13]:
#common function for both spanish and german

class split_pairs:
 def split_pairs_method(self,df1):
  text_pairs=[]

  for i in range(len(df1)):
    english, language = df1["english"][i], str(df1["sinhala"][i])  # Convert to string
    language="[start] " + language + " [end]"
    text_pairs.append((english,language))

  return text_pairs


In [14]:
#randomly selecting that if the above function work
class random_pair_test:
  def random_test_method(self,text_pairs):
   for i in range(3):
    print(random.choice(text_pairs))


In [15]:
#pairing for sinhala text

sinhala_text_pairs=split_pairs().split_pairs_method(df1)

random_pair_test().random_test_method(sinhala_text_pairs)

('Within days, Japan captured the American island of Guam.', '[start] දින කිහිපයක් ඇතුළත ජපානය ඇමරිකානු ගුවාම් විසින් ග්රහණය කර ගත්තේය. [end]')
('Love is blind.', '[start] ආදරය අන්ධයි. [end]')
("Tom is looking for someone to take Mary's place.", '[start] ටොම් මරියාගේ ස්ථානය ගැනීමට කෙනෙකු සොයමින් සිටී. [end]')


# Randomizing the data

In [16]:
random.shuffle(sinhala_text_pairs)

# Split the data into training, validation,testing

In [17]:
#class for splitting text pairs in to train,test,validation
class splitting:
    def splitting_method(self,text_pairs):
        num_val_sample=int(0.15*len(text_pairs))
        num_train_samples=len(text_pairs) - 2 * num_val_sample
        train_pairs=text_pairs[:num_train_samples]
        val_pairs=text_pairs[num_train_samples:num_train_samples+num_val_sample]
        test_pairs=text_pairs[num_train_samples+num_val_sample:]

        print("Total Sentences: ",len(text_pairs))
        print("Training set size: ",len(train_pairs))
        print("Validation set size: ",len(val_pairs))
        print("Testng set size: ",len(test_pairs))
        return train_pairs,val_pairs,test_pairs



In [18]:
sinhala_train_pairs,sinhala_val_pairs,sinhala_test_pairs=splitting().splitting_method(sinhala_text_pairs)


Total Sentences:  28906
Training set size:  20236
Validation set size:  4335
Testng set size:  4335


In [19]:
print(len(sinhala_test_pairs)+len(sinhala_train_pairs)+len(sinhala_val_pairs))

28906


In [20]:
print(sinhala_val_pairs[200])

('Not only were we hungry, but we were also suffering from thirst.', '[start] අපට බඩගිනවා පමණක් නොව, අපි ද පිපාසයෙන් පෙළුණා. [end]')


# Removing Puncuations

In [21]:
strip_chars = string.punctuation + "¿"
strip_chars = strip_chars.replace("[", "")
strip_chars = strip_chars.replace("]", "")


f"[{re.escape(strip_chars)}]"

f"{5+3}"

'8'

# Vectorizing the English and Sinhala text pairs

In [22]:
def custom_standardization(input_string):
    lowercase=tf.strings.lower(input_string)
    return tf.strings.regex_replace(lowercase,f"[{re.escape(strip_chars)}]", "")

vocab_size=15000
sequence_length=20

source_vectorization=layers.TextVectorization(
    max_tokens=vocab_size,
    output_mode="int",
    output_sequence_length=sequence_length,
)

target_vectorization=layers.TextVectorization(
    max_tokens=vocab_size,
    output_mode="int",
    output_sequence_length=sequence_length+1,
    standardize=custom_standardization,
)

train_english_texts=[pair[0] for pair in sinhala_train_pairs]
train_sinhala_texts=[pair[1] for pair in sinhala_train_pairs]

source_vectorization.adapt(train_english_texts)
target_vectorization.adapt(train_sinhala_texts)

In [23]:
print(train_english_texts[1])
print(train_sinhala_texts[1])

print(source_vectorization)

This is the same car that was left at the scene of the crime.
[start] අපරාධය ස්ථානයේ ඉතිරිව ඇති එකම මෝටර් රථය මෙයයි. [end]


In [24]:
print(sinhala_train_pairs[1:5])

[('This is the same car that was left at the scene of the crime.', '[start] අපරාධය ස්ථානයේ ඉතිරිව ඇති එකම මෝටර් රථය මෙයයි. [end]'), ('I watched you.', '[start] මම ඔයාව බැලුවා. [end]'), ("I was going to try to get into Tokyo University, but I've changed my mind.", '[start] මම ටෝකියෝ විශ්ව විද්යාලයට ඇතුල් වීමට උත්සාහ කළා, නමුත් මම මගේ අදහස වෙනස් කළා. [end]'), ('I hate Sundays.', '[start] මම ඉරිදාට වෛර කරනවා. [end]')]


# Preparing datasets for the translation task

In [25]:
batch_size=64

def format_dataset(eng,ger):
    eng=source_vectorization(eng)
    ger=target_vectorization(ger)
    return ({"english":eng,
             "german":ger[:,:-1],
             },ger[:,1:])

def make_dataset(pairs):
    eng_texts,ger_texts=zip(*pairs)
    eng_texts=list(eng_texts)
    ger_texts=list(ger_texts)
    dataset=tf.data.Dataset.from_tensor_slices((eng_texts,ger_texts))
    dataset=dataset.batch(batch_size)
    dataset=dataset.map(format_dataset,num_parallel_calls=4)

    return dataset.shuffle(2048).prefetch(16).cache()

train_data=make_dataset(sinhala_train_pairs)
val_data=make_dataset(sinhala_val_pairs)

# print(train_data)
# print(val_data)

for inputs, targets in train_data.take(1):
    print(f"inputs['english'].shape: {inputs['english'].shape}")
    print(f"inputs['german'].shape: {inputs['german'].shape}")
    print(f"targets.shape: {targets.shape}")



inputs['english'].shape: (64, 20)
inputs['german'].shape: (64, 20)
targets.shape: (64, 20)


In [26]:
print(list(train_data.as_numpy_iterator())[50])

({'english': array([[  15, 1015,  139, ...,    0,    0,    0],
       [  25, 2512,   30, ...,    0,    0,    0],
       [   5,   14,  908, ...,    0,    0,    0],
       ...,
       [   3,  105,  727, ...,    0,    0,    0],
       [   3,   40,   15, ...,    0,    0,    0],
       [   5,  659,    2, ...,    0,    0,    0]]), 'german': array([[   2,   10, 1284, ...,    0,    0,    0],
       [   2,   14,    7, ...,    0,    0,    0],
       [   2,    5, 4763, ...,    0,    0,    0],
       ...,
       [   2,    4, 3296, ...,    0,    0,    0],
       [   2,    6,   10, ...,    0,    0,    0],
       [   2,    5, 4427, ...,    0,    0,    0]])}, array([[  10, 1284, 3492, ...,    0,    0,    0],
       [  14,    7,  169, ...,    0,    0,    0],
       [   5, 4763, 2424, ...,    0,    0,    0],
       ...,
       [   4, 3296,    3, ...,    0,    0,    0],
       [   6,   10, 3150, ...,    0,    0,    0],
       [   5, 4427, 2054, ...,    0,    0,    0]]))


# Transformers encoder implemented as a subclassed Layer

In [27]:
class TransformerEncoder(layers.Layer):
    def __init__(self,embed_dim,dense_dim,num_heads,**kwargs):
        super().__init__(**kwargs)
        self.embed_dim=embed_dim
        self.dense_dim=dense_dim
        self.num_heads=num_heads
        self.attention=layers.MultiHeadAttention(
           num_heads=num_heads,key_dim=embed_dim)
        self.dense_proj=keras.Sequential(
            [layers.Dense(dense_dim,activation="relu"),
             layers.Dense(embed_dim),]
        )
        self.layernorm_1=layers.LayerNormalization()
        self.layernorm_2=layers.LayerNormalization()

    def call(self,inputs,mask=None):
        if mask is not None:
            mask=mask[:,tf.newaxis,:]
        attention_output=self.attention(
            inputs,inputs,attention_mask=mask
        )
        project_input=self.layernorm_1(inputs+attention_output)
        project_output=self.dense_proj(project_input)

        return self.layernorm_2(project_input+project_output)

    def get_config(self):
        config=super().get_config()
        config.update({
            "embed_dim": self.embed_dim,
            "num_heads": self.num_heads,
            "dense_dim": self.dense_dim,
        })
        return config

# Transformer decorder

In [28]:
class TransformerDecoder(layers.Layer):
    def __init__(self,embed_dim,dense_dim,num_heads,**kwargs):
        super().__init__(**kwargs)
        self.embed_dim=embed_dim
        self.dense_dim=dense_dim
        self.num_heads=num_heads
        self.attention_1=layers.MultiHeadAttention(
            num_heads=num_heads,key_dim=embed_dim)
        self.attention_2=layers.MultiHeadAttention(
            num_heads=num_heads,key_dim=embed_dim)
        self.dense_proj=keras.Sequential(
            [layers.Dense(dense_dim,activation="relu"),
             layers.Dense(embed_dim),]
        )
        self.layernorm_1=layers.LayerNormalization()
        self.layernorm_2=layers.LayerNormalization()
        self.layernorm_3=layers.LayerNormalization()
        self.supports_masking=True

    def get_config(self):
        config=super().get_config()
        config.update({
            "embed_dim": self.embed_dim,
            "num_heads": self.num_heads,
            "dense_dim": self.dense_dim,
        })
        return config

    def get_casual_attention_mask(self,inputs):
        input_shape=tf.shape(inputs)
        batch_size,sequence_length=input_shape[0],input_shape[1]
        i=tf.range(sequence_length)[:,tf.newaxis]
        j=tf.range(sequence_length)
        mask=tf.cast(i>=j,dtype="int32")
        mask=tf.reshape(mask,(1,input_shape[1],input_shape[1]))
        mult=tf.concat(
            [tf.expand_dims(batch_size,-1),
             tf.constant([1,1],dtype=tf.int32)],axis=0)

        return tf.tile(mask,mult)


    def call(self,inputs,encorder_outputs,mask=None):
        casual_mask=self.get_casual_attention_mask(inputs)
        if mask is not None:
            padding_mask=tf.cast(
                mask[:,tf.newaxis,:],dtype="int32"
            )
            padding_mask=tf.minimum(padding_mask,casual_mask)
        else:
            padding_mask=mask
        attention_output_1=self.attention_1(
            query=inputs,
            value=inputs,
            key=inputs,
            attention_mask=casual_mask
        )
        attention_output_1=self.layernorm_1(inputs+attention_output_1)
        attention_output_2=self.attention_2(
            query=attention_output_1,
            value=encorder_outputs,
            key=encorder_outputs,
            attention_mask=padding_mask,
        )
        attention_output_2=self.layernorm_2(
            attention_output_1+attention_output_2
        )
        proj_output=self.dense_proj(attention_output_2)
        return self.layernorm_3(attention_output_2+proj_output)




# Positional Encoding

In [29]:
class PositionalEmbedding(layers.Layer):
    def __init__(self,sequence_length,input_dim,output_dim,**kwargs):
        super().__init__(**kwargs)
        self.token_embeddings=layers.Embedding(
            input_dim=input_dim,output_dim=output_dim)
        self.position_embeddings=layers.Embedding(
            input_dim=sequence_length,output_dim=output_dim)
        self.sequence_length=sequence_length
        self.input_dim=input_dim
        self.output_dim=output_dim

    def call(self,inputs):
        length=tf.shape(inputs)[-1]
        positions=tf.range(start=0,limit=length,delta=1)
        embedded_tokens=self.token_embeddings(inputs)
        embedded_positions=self.position_embeddings(positions)

        return embedded_tokens + embedded_positions


    def compute_mask(self, inputs, mask=None):
        return tf.not_equal(inputs, 0)

    def get_config(self):
        config=super(PositionalEmbedding,self).get_config()
        config.update({
            "output_dim": self.output_dim,
            "sequence_length": self.sequence_length,
            "input_dim": self.input_dim,
        })
        return config

In [30]:
# from tensorflow.python.framework.ops import disable_eager_execution
# disable_eager_execution()

# End-to-End Transformer

In [53]:
embed_dim=256
dense_dim=2048
num_heads=8

encoder_inputs=keras.Input(shape=(None,), dtype="int64", name="english")
x=PositionalEmbedding(sequence_length,vocab_size,embed_dim)(encoder_inputs)
encoder_outputs=TransformerEncoder(embed_dim,dense_dim,num_heads)(x)

decorder_inputs=keras.Input(shape=(None,),dtype="int64",name="sinhala")
x=PositionalEmbedding(sequence_length,vocab_size,embed_dim)(decorder_inputs)
x=TransformerDecoder(embed_dim,dense_dim,num_heads)(x,encoder_outputs)
x=layers.Dropout(0.5)(x)
decorder_outputs=layers.Dense(vocab_size,activation="softmax")(x)
transformer=keras.Model([encoder_inputs,decorder_inputs],decorder_outputs)

# Training the sequence-to-sequence Transformer

In [54]:
transformer.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 english (InputLayer)        [(None, None)]               0         []                            
                                                                                                  
 sinhala (InputLayer)        [(None, None)]               0         []                            
                                                                                                  
 positional_embedding_2 (Po  (None, None, 256)            3845120   ['english[0][0]']             
 sitionalEmbedding)                                                                               
                                                                                                  
 positional_embedding_3 (Po  (None, None, 256)            3845120   ['sinhala[0][0]']       

In [33]:
transformer.compile(
    optimizer="rmsprop",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

transformer.fit(train_data,epochs=50,validation_data=val_data)

Epoch 1/50
317/317 [==============================] - 39s 96ms/step - loss: 5.5004 - accuracy: 0.3130 - val_loss: 4.7711 - val_accuracy: 0.3683
Epoch 2/50
317/317 [==============================] - 21s 66ms/step - loss: 4.4595 - accuracy: 0.3886 - val_loss: 4.2214 - val_accuracy: 0.4160
Epoch 3/50
317/317 [==============================] - 21s 65ms/step - loss: 3.9244 - accuracy: 0.4387 - val_loss: 3.9326 - val_accuracy: 0.4483
Epoch 4/50
317/317 [==============================] - 21s 66ms/step - loss: 3.5416 - accuracy: 0.4759 - val_loss: 3.7422 - val_accuracy: 0.4704
Epoch 5/50
317/317 [==============================] - 22s 68ms/step - loss: 3.2268 - accuracy: 0.5081 - val_loss: 3.6573 - val_accuracy: 0.4819
Epoch 6/50
317/317 [==============================] - 22s 68ms/step - loss: 2.9708 - accuracy: 0.5361 - val_loss: 3.5583 - val_accuracy: 0.4930
Epoch 7/50
317/317 [==============================] - 22s 69ms/step - loss: 2.7620 - accuracy: 0.5617 - val_loss: 3.5750 - val_accuracy:

In [48]:
import numpy as np

sinahala_vocab = target_vectorization.get_vocabulary()
sinahala_index_lookup = dict(zip(range(len(sinahala_vocab)), sinahala_vocab))
max_decoded_sentence_length = 20

def decode_sequence(input_sentence):
    tokenized_input_sentence = source_sentence = source_vectorization([input_sentence])
    decoded_sentence = "[start]"

    for i in range(max_decoded_sentence_length):
        tokenized_target_sentence = target_vectorization([decoded_sentence])[:, :-1]
        predictions = transformer([tokenized_input_sentence, tokenized_target_sentence])
        sampled_token_index = np.argmax(predictions[0, i, :])
        sampled_token = sinahala_index_lookup[sampled_token_index]
        decoded_sentence += " " + sampled_token
        if sampled_token == "[end]":
            break

    return decoded_sentence

In [49]:
ger_eng_texts = [pair[0] for pair in sinhala_test_pairs]
for _ in range(20):
    input_sentence = random.choice(ger_eng_texts)
    print("-")
    print(input_sentence)
    print(decode_sequence(input_sentence))

-
What have you got?
[start] ඔබට ඇත්තේ කුමක්ද [end]
-
Cheating on one's spouse is not usually considered acceptable behavior.
[start] සීයා සහ කකුල් විවාහ ගිවිසගෙන සිටී [end]
-
Tom finally realized there was something wrong.
[start] ටොම් ඉතා භයානක දෙයක් හරි බවට පත්විය [end]
-
Always keep a bucket of water handy, in case of fire.
[start] සෑම විටම මා සතුව ඇති එය වහාම ජලය හිස් අවශ්යය [end]
-
I wish I could do everything that I wanted to do.
[start] මට අවශ්ය නම් මට කළ හැකි කිරීම සඳහා එය කිරීම හොඳයි [end]
-
It was like that.
[start] එය ඒ වගේ [end]
-
Since I'm not so good at swimming, I avoid swimming in water that's over my head.
[start] මම හොඳයි එය බේස්බෝල් බැලීමට තරම් කුඩා බැවින් මට කාලගුණය සීතල වූ නිසා මම හිතුවේ නැහැ [end]
-
No matter how many people tell him he made a mistake, Tom still insists he's right.
[start] ටොම් කොතරම් විභාගය ඔහු කිසි විටෙකත් භයානක දෙයක් පැවසිය යුතු බව ඔහු පෙනී සිටිනු ඇත [end]
-
I waited half an hour for my friend, but he didn't turn up.
[start] ඔහු මගේ මිතුරෙකු ප

In [36]:
# from google.colab import drive
# drive.mount('/content/drive')

Mounted at /content/drive


In [37]:
# transformer.save('/content/drive/My Drive/Translators')



# New 20 Observations


In [50]:
custom_english_sentences = ["Wheat is the primary grain cultivated in the Midwest.",
"Would you mind dropping by my cubicle when you get a chance?",
"Received any New Year's gifts from Sarah this time around?",
"Have you caught wind of the incident involving Tom and Mary?",
"Embrace uncertainty, but always stay prepared for it.",
"Make the best out of what you possess, wherever you may be.",
"Can you pick up on any sounds coming from the adjacent room?",
"Aware of the fine line between right and wrong?",
"Do you believe she's entirely innocent in this matter?",
"Does the level of humidity impact plant growth significantly?",
"Remember your raincoat when stepping out; the forecast is uncertain.",
"Avoid delaying tasks; accomplish what's within your reach today.",
"Keep out of other people's affairs; it's not your place.",
"Rest assured, Tom can handle himself just fine.",
"No need to worry about my dog; he poses no threat to you.",
"Exercise caution while driving, lest you meet with an accident.",
"Endured many hardships during the turbulent years of conflict.",
"This image triggers memories of my father each time I gaze upon it.",
"Distinguishable by their distinctive features, elephants belong to three distinct species.",
"Elephants rank as the largest terrestrial animals on the planet."]

In [51]:
print(custom_english_sentences[0])

Wheat is the primary grain cultivated in the Midwest.


In [52]:
for sentence in custom_english_sentences:
    print("Original English Sentence:", sentence)
    translated_sentence = decode_sequence(sentence)
    print("Translated Sinhala Sentence:", translated_sentence)
    print("-" * 20)  # Separator

Original English Sentence: Wheat is the primary grain cultivated in the Midwest.
Translated Sinhala Sentence: [start] නැව විශාල අයිස් න එක විශේෂ විහිළුවක් [end]
--------------------
Original English Sentence: Would you mind dropping by my cubicle when you get a chance?
Translated Sinhala Sentence: [start] ඔබට නිවාඩුවක් දිය හැකි විට ඔබේ ඇඳුම් ඔබට දිය යුතුද [end]
--------------------
Original English Sentence: Received any New Year's gifts from Sarah this time around?
Translated Sinhala Sentence: [start] මෙම නව යම් මුදලක් ඉල්ලා සිටීමට තීරණය කර ඇත [end]
--------------------
Original English Sentence: Have you caught wind of the incident involving Tom and Mary?
Translated Sinhala Sentence: [start] ඔබ සහ ටොම් මරියා කුඩා කළේ එයයි [end]
--------------------
Original English Sentence: Embrace uncertainty, but always stay prepared for it.
Translated Sinhala Sentence: [start] සඳහා දොර වසා දැමීමට අමතක කිරීම සඳහා එය නිතරම සිතමි [end]
--------------------
Original English Sentence: Make the best ou